In [2]:
from typing import List, Dict

import numpy as np
import pandas as pd
import glob
import os

from src.NoXi.preprocessing.labels_preprocessing import read_noxi_label_file

<h3>Labels loadings</h3>

In [ ]:
# engagement_novice(expert).annotation in the directory means that this file is a GOLD STANDARD
# engagement_novice(expert)_*name*.annotation in the directory means that this file is annotated by some *name* rater
# in the directory, it can be several engagement_novice(expert)_*name*.annotation files, meaning that the videofile was annotated by several raters

In [5]:
path_to_labels=r'D:\Databases\NoXi\NoXi_annotations'
gold_standard_files=glob.glob(os.path.join(path_to_labels,'**','**','engagement_novice.annotation~')) + glob.glob(os.path.join(path_to_labels,'**','**','engagement_expert.annotation~'))
all_files=glob.glob(os.path.join(path_to_labels,'**','**','*.annotation~'))


In [ ]:
# function for loading data
def load_annotation_files_by_paths(paths:List[str])->Dict[str, np.ndarray]:
    """Loads NoXi *annotation~ files using provided list of paths

    :param paths: List[str]
            Paths to files for laoding
    :return: List[np.ndarray]
            List of np.ndarrays with labels.
    """
    labels={}
    for path in paths:
        loaded_labels=read_noxi_label_file(path)
        labels[path]=loaded_labels
    return labels

In [7]:
gold_standard_files

['D:\\Databases\\NoXi\\NoXi_annotations\\English\\026_2016-04-06_Nottingham\\engagement_novice.annotation~',
 'D:\\Databases\\NoXi\\NoXi_annotations\\English\\027_2016-04-06_Nottingham\\engagement_novice.annotation~',
 'D:\\Databases\\NoXi\\NoXi_annotations\\English\\028_2016-04-06_Nottingham\\engagement_novice.annotation~',
 'D:\\Databases\\NoXi\\NoXi_annotations\\English\\030_2016-04-06_Nottingham\\engagement_novice.annotation~',
 'D:\\Databases\\NoXi\\NoXi_annotations\\English\\034_2016-04-07_Nottingham\\engagement_novice.annotation~',
 'D:\\Databases\\NoXi\\NoXi_annotations\\English\\052_2016-04-12_Nottingham\\engagement_novice.annotation~',
 'D:\\Databases\\NoXi\\NoXi_annotations\\French\\001_2016-03-17_Paris\\engagement_novice.annotation~',
 'D:\\Databases\\NoXi\\NoXi_annotations\\French\\002_2016-03-17_Paris\\engagement_novice.annotation~',
 'D:\\Databases\\NoXi\\NoXi_annotations\\French\\003_2016-03-17_Paris\\engagement_novice.annotation~',
 'D:\\Databases\\NoXi\\NoXi_annotatio